## Overview
Objective

Biome Generation  
1: input a seed and output a valid biome map  
2: given a biome map, generate a valid neighboring biome map  

Terrain Generation  
1: input noise, output biome chunk  
2: input set of chunks, output set of neighboring chunks  

In [13]:
import torch

In [82]:
class BiomeMap:
    def __init__(self, n, biomes, biome_rules):
        self.n = n
        self.n_biomes = len(biomes)
        self.map = torch.zeros([len(biomes), n, n], dtype=torch.float32)
        
        self.biome_ilookup = { idx: b for idx,b, in enumerate(biomes) }
        self.biome_nlookup = { b: idx for idx,b in self.biome_ilookup.items() }

    def inbounds(self, x, y):
        return x >= 0 and x < self.n and y >= 0 and y < self.n

    # Tensor biome
    def tbiome(self, x, y):
        return self.map[:, x, y] if self.inbounds(x, y) else None
    # Index biome
    def ibiome(self, x, y):
        idx, value = torch.max(self.tbiome(x, y), 0)
        return int(idx)
    # Name biome
    def nbiome(self, x, y):
        return self.biome_ilookup[self.ibiome(x, y)]

    # Relative tensor biome
    def r_tbiome(self, locx, locy, x, y):
        return self.biome(locx + x, locy + y)
    def r_ibiome(self, locx, locy, x, y):
        return self.ibiome(locx + x, locy + y)
    def r_nbiome(self, locx, locy, x, y):
        return self.nbiome(locx + x, locy + y)

    def s_ibiome(self, x, y, i):
        self.map[:, x, y] = torch.zeros([self.n_biomes], dtype=torch.float32)
        self.map[i, x, y] = 1.0
    def s_nbiome(self, x, y, biome_name):
        return self.s_ibiome(x, y, self.biome_nlookup[biome_name])

    def __str__(self):
        output = ''
        for i in range(self.n):
            for j in range(self.n):
                output += self.nbiome(i, j)[0]   # append first letter of biome
            output += '\n'
        return output
    

In [176]:
b = BiomeMap(n=5, biomes=['water', 'coast', 'land'], biome_rules=None)
b.map[0, :, :] = 1
b.s_nbiome(1, 1, 'coast')
b.s_nbiome(1, 2, 'coast')
print(b)
b.map[:, :, :]

ccccc
ccccc
ccccc
ccccc
ccccc



tensor([[[1., 1., 1., 1., 1.],
         [1., 0., 0., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]],

        [[0., 0., 0., 0., 0.],
         [0., 1., 1., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]])

In [179]:
# sum on dimension
def neighbor_sum(x, y, n, bmap):
    x0 = max(x-n, 0)
    xn = min(x+n+1, bmap.n)
    y0 = max(y-n, 0)
    yn = min(y+n+1, bmap.n)
    
#     print(bmap.map[:, x0:xn, y0:yn])
    
    return torch.sum(bmap.map[:, x0:xn, y0:yn], (1,2))

def coast_rule(x, y, bmap):
    water_idx = bmap.biome_nlookup['water']
    nsum = neighbor_sum(x, y, 1, bmap)
    return bool(nsum[water_idx] > 1 and nsum[water_idx] < 8)

In [180]:
print(neighbor_sum(x=1, y=1, n=1, bmap=b))
print('--')
print(coast_rule(1, 1, b))

tensor([7., 2., 0.])
--
True
